In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random
%matplotlib inline

In [22]:
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression
from sklearn import ensemble
from sklearn.svm import SVC, LinearSVC
from sklearn import metrics
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer

In [12]:
data_path = 'melb_data.csv'
data = pd.read_csv(data_path)
del data['Unnamed: 0']
data.head()

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,3/12/2016,2.5,3067.0,...,1.0,1.0,202.0,NaN,NaN,Yarra,-37.7996,144.9984,Northern Metropolitan,4019.0
1,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,4/02/2016,2.5,3067.0,...,1.0,0.0,156.0,79.0,1900.0,Yarra,-37.8079,144.9934,Northern Metropolitan,4019.0
2,Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,4/03/2017,2.5,3067.0,...,2.0,0.0,134.0,150.0,1900.0,Yarra,-37.8093,144.9944,Northern Metropolitan,4019.0
3,Abbotsford,40 Federation La,3,h,850000.0,PI,Biggin,4/03/2017,2.5,3067.0,...,2.0,1.0,94.0,NaN,NaN,Yarra,-37.7969,144.9969,Northern Metropolitan,4019.0
4,Abbotsford,55a Park St,4,h,1600000.0,VB,Nelson,4/06/2016,2.5,3067.0,...,1.0,2.0,120.0,142.0,2014.0,Yarra,-37.8072,144.9941,Northern Metropolitan,4019.0


In [18]:
print(data.columns)
features = ['Suburb', 'Address', 'Rooms', 'Type', 'Method', 'SellerG',
       'Date', 'Distance', 'Postcode', 'Bedroom2', 'Bathroom', 'Car',
       'Landsize', 'BuildingArea', 'YearBuilt', 'CouncilArea', 'Lattitude',
       'Longtitude', 'Regionname', 'Propertycount']

x = data[features]
y = data.Price

x_train , x_valid , y_train , y_valid = train_test_split(x,y,test_size=0.3)

Index(['Suburb', 'Address', 'Rooms', 'Type', 'Price', 'Method', 'SellerG',
       'Date', 'Distance', 'Postcode', 'Bedroom2', 'Bathroom', 'Car',
       'Landsize', 'BuildingArea', 'YearBuilt', 'CouncilArea', 'Lattitude',
       'Longtitude', 'Regionname', 'Propertycount'],
      dtype='object')


## Data Preprocessing

In [19]:
data.isnull().sum()/len(data)*100

Suburb            0.000000
Address           0.000000
Rooms             0.000000
Type              0.000000
Price             0.000000
Method            0.000000
SellerG           0.000000
Date              0.000000
Distance          0.005436
Postcode          0.005436
Bedroom2         18.857360
Bathroom         18.868232
Car              19.439008
Landsize         26.054577
BuildingArea     57.806045
YearBuilt        51.304631
CouncilArea      33.501848
Lattitude        18.112633
Longtitude       18.112633
Regionname        0.005436
Propertycount     0.005436
dtype: float64

In [24]:
# cols with missing value 
cols_with_missing = [cols for cols in x_train.columns if x_train[cols].isnull().any() and x_train[cols].dtype == 'int64']

# making copy of data for imputation 
x_train_plus = x_train.copy()
x_valid_plus = x_valid.copy()

# Make new columns indicating what will be imputed
for col in cols_with_missing:
    x_train_plus[col + '_was_missing'] = x_train_plus[col].isnull()
    x_valid_plus[col + '_was_missing'] = x_valid_plus[col].isnull()

# Imputation
my_imputer = SimpleImputer()
imputed_x_train_plus = pd.DataFrame(my_imputer.fit_transform(x_train_plus))
imputed_x_valid_plus = pd.DataFrame(my_imputer.transform(x_valid_plus))

# Imputation removed column names; put them back
imputed_x_train_plus.columns = x_train_plus.columns
imputed_x_valid_plus.columns = x_valid_plus.columns

ValueError: Cannot use mean strategy with non-numeric data:
could not convert string to float: 'Yarraville'

In [16]:
# convert objects to int
s = (data.dtypes == 'object')
object_cols = list(s[s].index)

# Apply one-hot encoder to each column with categorical data
OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
OH_cols_train = pd.DataFrame(OH_encoder.fit_transform(imputed_x_train[object_cols]))
OH_cols_valid = pd.DataFrame(OH_encoder.transform(imputed_x_valid[object_cols]))

# One-hot encoding removed index; put it back
OH_cols_train.index = imputed_x_train.index
OH_cols_valid.index = imputed_x_valid.index

# Remove categorical columns (will replace with one-hot encoding)
num_x_train = imputed_x_train.drop(object_cols, axis=1)
num_x_valid = imputed_x_valid.drop(object_cols, axis=1)

# Add one-hot encoded columns to numerical features
OH_x_train = pd.concat([num_x_train, OH_cols_train], axis=1)
OH_x_valid = pd.concat([num_x_valid, OH_cols_valid], axis=1)


ValueError: Input contains NaN